# Compute mean and std arrival for train_type, hour and station (~20 minutes runtime)

In [1]:
%%configure
{"conf": {
    "spark.app.name": "dslab-group_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6481,application_1589299642358_0970,pyspark,idle,Link,Link,
6485,application_1589299642358_0974,pyspark,idle,Link,Link,
6487,application_1589299642358_0976,pyspark,idle,Link,Link,
6490,application_1589299642358_0979,pyspark,idle,Link,Link,
6493,application_1589299642358_0982,pyspark,idle,Link,Link,
6500,application_1589299642358_0989,pyspark,idle,Link,Link,
6501,application_1589299642358_0990,pyspark,idle,Link,Link,
6502,application_1589299642358_0991,pyspark,busy,Link,Link,
6503,application_1589299642358_0992,pyspark,idle,Link,Link,


#### Imports:

In [2]:
import networkx as nx
from geopy.distance import distance as geo_distance
from pyspark.sql import Row
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
from networkx.algorithms.shortest_paths.weighted import dijkstra_path

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6504,application_1589299642358_0993,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Compute mean and std arrival delay

In [9]:
stops_in_zurich = stops_zurich.select('stop_id').distinct().rdd.flatMap(lambda x:x).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Taken from find_train_type_correspondace.ipynb
replace_actual = {
    'BUS': 'Bus', # Buses
    'B': 'Bus',
    'NFB': 'Bus',
    'KB': 'Bus',
    'BAT': 'Bus',
    'Trm': 'Tram', # Trams
    'T': 'Tram',
    'TRAM': 'Tram',
    'ATZ': 'ARZ', #AutoZug
    'D': 'RE', # Regional
    'RB': 'R',
    'M': 'Metro', # Metro
    'ICE': 'IC', # InterCityExpress, but routes.txt doesn't have that category
    'IRE': 'IR', # InterRegioExpress, but routes.txt doesn't have that category
    'BN': '', # Night
    'TN': '',
    'SN': '',
    'BT': '',
    'VAE': '', # Panorama trains in the Alps
    'PE': '',
    'TER': '', # France
    'TE2': '',
    'RJX': '', # International
    'null': '' # Other
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
@udf("string")
def replace_verkehrsmittel_text(text):
    if text in replace_actual.keys():
        return replace_actual[text]
    else:
        return text

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
actual = spark.read.format('orc').load('/data/sbb/orc/istdaten/')\
                                 .where(col('bpuic').isin(stops_in_zurich))\
                                 .where((col("ankunftszeit") != "") & (col("an_prognose") != "") & (col("verkehrsmittel_text") != ""))\
                                 .select(col('bpuic').alias('stop_id'),\
                                         replace_verkehrsmittel_text(col('verkehrsmittel_text')).alias('verkehrsmittel_text'),\
                                         from_unixtime(unix_timestamp('ankunftszeit', 'dd.MM.yyy HH:mm')).alias('ankunftszeit'),\
                                         from_unixtime(unix_timestamp('an_prognose', 'dd.MM.yyy HH:mm:ss')).alias('an_prognose'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
actual.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+-------------------+-------------------+
|stop_id|verkehrsmittel_text|       ankunftszeit|        an_prognose|
+-------+-------------------+-------------------+-------------------+
|8503000|                 EC|2018-09-03 21:51:00|2018-09-03 21:53:40|
|8503000|                 EC|2018-09-03 10:51:00|2018-09-03 10:51:28|
|8503000|                 IC|2018-09-03 07:00:00|2018-09-03 07:00:01|
|8503000|                 IC|2018-09-03 21:23:00|2018-09-03 21:24:55|
|8503000|                 IC|2018-09-03 08:26:00|2018-09-03 08:28:06|
|8503000|                 IC|2018-09-03 17:53:00|2018-09-03 17:55:21|
|8503000|                 IC|2018-09-03 10:00:00|2018-09-03 09:59:07|
|8503000|                 EC|2018-09-03 12:51:00|2018-09-03 12:52:50|
|8503000|                 IC|2018-09-03 08:10:00|2018-09-03 08:15:32|
|8503000|                 IC|2018-09-03 17:10:00|2018-09-03 17:10:06|
|8503000|                 IC|2018-09-03 18:10:00|2018-09-03 18:09:46|
|8503000|           

In [14]:
actual = actual.withColumn('hour', hour(col('ankunftszeit')))\
               .withColumn('diff', unix_timestamp('an_prognose') - unix_timestamp('ankunftszeit'))\
               .select(col('stop_id'), col('verkehrsmittel_text'), col('hour'), col('diff'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
actual.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+----+----+
|stop_id|verkehrsmittel_text|hour|diff|
+-------+-------------------+----+----+
|8503000|                 EC|  21| 160|
|8503000|                 EC|  10|  28|
|8503000|                 IC|   7|   1|
|8503000|                 IC|  21| 115|
|8503000|                 IC|   8| 126|
|8503000|                 IC|  17| 141|
|8503000|                 IC|  10| -53|
|8503000|                 EC|  12| 110|
|8503000|                 IC|   8| 332|
|8503000|                 IC|  17|   6|
|8503000|                 IC|  18| -14|
|8503000|                 IC|   7| 118|
|8503000|                 IC|   8| 112|
|8503000|                 IC|   6| 120|
|8503000|                 IC|   7| 167|
|8503000|                 IC|   7| -20|
|8503000|                 IC|   8|  90|
|8503000|                 IC|   8| -59|
|8503000|                 IC|   9|  48|
|8503000|                 IC|   9| 916|
+-------+-------------------+----+----+
only showing top 20 rows

In [16]:
actual = actual.groupBy('stop_id', 'verkehrsmittel_text', 'hour').agg(mean("diff").alias("mean"), stddev("diff").alias("std"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
actual = actual.na.fill(0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
actual.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[stop_id: string, verkehrsmittel_text: string, hour: int, mean: double, std: double]

In [19]:
actual.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+----+-------------------+------------------+
|stop_id|verkehrsmittel_text|hour|               mean|               std|
+-------+-------------------+----+-------------------+------------------+
|8503202|                 RE|  10|   96.5210970464135| 89.59750827144028|
|8503000|                 RE|  20|  14.40864255519023| 94.10998924475062|
|8502277|                 RE|   7|-12.576200417536535| 44.81269640129601|
|8502188|                  S|   0|  65.93168880455408| 116.4165437465676|
|8591252|               Tram|  15| 46.415761962203455|154.85775842747395|
|8591138|               Tram|  19| 41.096322241681264| 115.7428796557576|
|8591381|               Tram|  23|  75.40832557139787|113.17642398065263|
|8576195|               Tram|  17|   67.1224444690021|197.00756086679405|
|8591051|               Tram|  18| 63.364261543499616| 169.9632979451372|
|8591236|               Tram|   8|  54.13117152261368| 75.04036210482607|
|8503083|               Tram|  11|  59

## Save to local

In [20]:
%%spark -o actual -n 1000000

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
%%local
actual.to_pickle('pickle_delay_distribution')

In [22]:
actual.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

36950

## Load from local

In [23]:
%%local
import pandas as pd
actual = pd.read_pickle('pickle_delay_distribution')

In [24]:
%%send_to_spark -i actual -t df -m 1000000

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'test' as 'test' to Spark kernel

In [25]:
actual.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

36950

In [27]:
actual.select(col('verkehrsmittel_text')).distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|verkehrsmittel_text|
+-------------------+
|                 IC|
|                ZUG|
|                 RE|
|                 NJ|
|                TGV|
|                 EN|
|                 IR|
|                 EC|
|               Tram|
|                 AG|
|                  S|
|                  R|
|                Bus|
|                EXT|
|                  P|
|                 RJ|
|                   |
|                 NB|
+-------------------+

# Temp: should pickle stop_times_zurich

In [3]:
# Loading data, these are snapshots of the all available data
# Calendar and trips are useful to filter the other dataframe according to the day

stop_times = spark.read.format('orc').load('/data/sbb/timetables/orc/stop_times/000000_0')
stops = spark.read.format('orc').load('/data/sbb/timetables/orc/stops/000000_0')
trips = spark.read.format('orc').load('/data/sbb/timetables/orc/trips/000000_0')
calendar = spark.read.format('orc').load('/data/sbb/timetables/orc/calendar/000000_0')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2. Pre-processing: 

Here, we pre-process the data to correspond to the following criteria:
- only consider journeys at reasonable hours of the day, and on a typical business day, and assuming the schedule of May 13-17, 2019
- allow short (max 500m "As the Crows Flies") walking distances for transfers between two stations, and assume a walking speed of 50m/1min on a straight line, regardless of obstacles, human-built or natural, such as building, highways, rivers, or lakes
- only consider journeys that start and end on known station coordinates (train station, bus stops, etc.), never from a random location
- only consider stations in a 15km radius of Zürich's train station, Zürich HB (8503000), (lat, lon) = (47.378177, 8.540192)
- only consider stations in the 15km radius that are reachable from Zürich HB, either directly, or via transfers through other stations within the area
- assume that the timetables remain unchanged throughout the 2018 - 2019 period

Filtering criteria for later use (e.g. probability intervals and planner):
- assuming that delays or travel times on the public transport network are uncorrelated with one another
- once a route is computed, a traveller is expected to follow the planned routes to the end, or until it fails (i.e. miss a connection)
- planner will not need to mitigate the traveller's inconvenience if a plan fails

### Stop times during rush-hour: 

Only consider journeys at reasonable hours of the day, thus we take only stop times that are in the window of rush-hour (e.g. from 8 a.m. to 8 p.m.). 

In [4]:
# Filter stop_times to be only in 08:00-19:59:
stop_times = stop_times.where((col('departure_time') >= '08:00:00') 
                              & (col('departure_time') <= '19:59:59'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Stations around Zürich HB:

Only consider stations in a 15km radius of Zürich's train station (Zürich HB). 

First we get the geolocation of Zürich Hauptbahnhof to be able to calculate the distance of the other stations to the Hauptbahnhof. 

In [5]:
zurich_pos = stops.where(column('stop_name') == 'Zürich HB').select('stop_lat', 'stop_lon').collect()
zurich_pos = (zurich_pos[0][0], zurich_pos[0][1])
print('Location of Zürich Hauptbahnhof (lat, lon) :'+str(zurich_pos))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Location of Zürich Hauptbahnhof (lat, lon) :(47.3781762039461, 8.54019357578468)

In [6]:
@udf("float")
def compute_distance(x1, y1, x2, y2):
    return geo_distance((x1, y1), (x2,y2)).m

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def zurich_distance(x, y):
    """zurich_distance: returns the distance of a station to Zurich HB
    @input: (lat,lon) of a station
    @output: distance in km to Zurich HB
    """
    return geo_distance(zurich_pos, (x,y)).km

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Then we create a dataframe `stops_zurich` of the stations where we add a column for the distance to Zurich HB. In that dataframe, we keep only those that are in a radisu of 15km to the HB. The same filter is applied to the `stop_times` df mentioned above. 

In [8]:
# filter stops:
stops_distance = stops.rdd.map(lambda x: (x['stop_id'], zurich_distance(x['stop_lat'], x['stop_lon'])))
stops_distance = spark.createDataFrame(stops_distance.map(lambda r: Row(stop_id=r[0], 
                                                                        zurich_distance=r[1])))

stops_distance = stops_distance.filter(column('zurich_distance') <= 15)

#print('There are '+str(stops_distance.count())+' stops in a radius of 15km around Zurich HB')

# add distance to HB to stops info and keep only in radius of 15km
stops_zurich = stops_distance.join(stops, on='stop_id')

# keep only stop times in radius of 15km of Zurich
stop_times_zurich = stop_times.join(stops_distance.select('stop_id'), on='stop_id')

#print('There are '+str(stop_times_zurich.count())+ ' stop times in a radius of 15km around  Zurich HB')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…